## ActiveLoop Course - LangChain & Vector Databases in Production

> https://learn.activeloop.ai/enrollments

---

In [1]:
import sys
sys.path.append("/Users/shaunaksen/Documents/personal-projects/Natural-Language-Processing/LLM Concepts/mini_projects/chatgpt_clone")

In [2]:
def print_wrap(text, width=100):
 """
 Prints text wrapped to a set width.

 Args:
   text: The text to wrap.
   width: The maximum width of each line.

 Returns:
   None
 """
 words = text.split()
 current_line = ""
 for word in words:
   if len(current_line) + len(word) + 1 > width:
     print(current_line)
     current_line = word
   else:
     current_line += " " + word
 if current_line:
   print(current_line)

In [3]:
import os

# models and embeddings
from langchain.chat_models import AzureChatOpenAI
from langchain.llms import AzureOpenAI, HuggingFaceHub
from langchain.embeddings import AzureOpenAIEmbeddings

# prompts and utils
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

# memory
from langchain.memory import ConversationBufferMemory

# chains
from langchain.chains import LLMChain, ConversationChain, RetrievalQA

# vectorstores
from langchain.vectorstores import DeepLake, FAISS
from langchain.vectorstores.chroma import Chroma

# agents and tools
from langchain.agents import initialize_agent, Tool, load_tools
from langchain.agents import AgentType
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.callbacks import get_openai_callback

from creds import (AZURE_API_BASE, AZURE_API_KEY, AZURE_API_VERSION, 
                   ACTIVELOOP_TOKEN, GOOGLE_API_KEY, GOOGLE_CSE_ID,
                   HUGGINGFACEHUB_API_TOKEN)


In [4]:
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HUGGINGFACEHUB_API_TOKEN

In [5]:
def init_models() -> list:
    embeddings = AzureOpenAIEmbeddings(
        deployment="text-embedding-ada-002",
        model="text-embedding-ada-002",
        openai_api_type='azure',
        azure_endpoint=AZURE_API_BASE,
        openai_api_key=AZURE_API_KEY,
        openai_api_version=AZURE_API_VERSION,
        chunk_size=1, max_retries=1e0 
    )

    llm_chat_gpt_4 = AzureChatOpenAI(deployment_name='gpt-4-32k',
                          model='gpt-4-32k',
                          openai_api_type='azure',
                          azure_endpoint=AZURE_API_BASE,
                          openai_api_key=AZURE_API_KEY,
                          openai_api_version=AZURE_API_VERSION,
                          max_retries=2,
                          temperature=0,
                          streaming=True
                          )

    llm_gpt_4 = AzureOpenAI(deployment_name='gpt-4-32k',
                            model='gpt-4-32k',
                            openai_api_type='azure',
                            azure_endpoint=AZURE_API_BASE,
                            openai_api_key=AZURE_API_KEY,
                            openai_api_version=AZURE_API_VERSION,
                            max_retries=2,
                            temperature=0,
                            streaming=True
                            )
    
    llm_text_davinci = AzureOpenAI(deployment_name='text-davinci-003',
                          model='text-davinci-003',
                          openai_api_type='azure',
                          azure_endpoint=AZURE_API_BASE,
                          openai_api_key=AZURE_API_KEY,
                          openai_api_version=AZURE_API_VERSION,
                          max_retries=2,
                          temperature=0.5)
    
    hub_llm = HuggingFaceHub(
        repo_id="google/flan-t5-large",
        model_kwargs={"temperature": 0}
    )
    
    return [embeddings, llm_chat_gpt_4, llm_gpt_4, llm_text_davinci, hub_llm]

In [6]:
embeddings, llm_chat_gpt_4, llm_gpt_4, llm_text_davinci, hub_llm = init_models()

/Users/shaunaksen/miniconda3/envs/aiops_venv/lib/python3.9/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'InferenceApi' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [6]:
text = "Suggest a personalized workout routine for someone looking to improve cardiovascular endurance and prefers outdoor activities."
print(llm_text_davinci(text))



1. Start with a warm-up of 5-10 minutes of light jogging and dynamic stretching.

2. Perform a HIIT workout - alternating between sprinting for 30 seconds and jogging for 90 seconds for a total of 20 minutes.

3. Follow up with a 7-10 minute jog at a moderate pace.

4. Finish with 5-10 minutes of stretching and cool down.

5. Repeat this workout 3-4 times a week.

6. On the other days, go for a 30-60 minute walk or jog at a moderate pace.

7. Mix it up by adding in outdoor activities such as biking, swimming, or rowing.


## Few-shot learning

Few-shot learning is a remarkable ability that allows LLMs to learn and generalize from limited examples. Prompts serve as the input to these models and play a crucial role in achieving this feature. With LangChain, examples can be hard-coded, but dynamically selecting them often proves more powerful, enabling LLMs to adapt and tackle tasks with minimal training data swiftly.

This approach involves using the `FewShotPromptTemplate` class, which takes in a `PromptTemplate` and a list of a few shot examples. The class formats the prompt template with a few shot examples, which helps the language model generate a better response. We can streamline this process by utilizing LangChain's FewShotPromptTemplate to structure the approach:

In [7]:
# create our examples
examples = [
    {
        "query": "What's the weather like?",
        "answer": "It's raining cats and dogs, better bring an umbrella!"
    }, {
        "query": "How old are you?",
        "answer": "Age is just a number, but I'm timeless."
    }
]

In [8]:
# create an example template
example_template = """
User: {query}
AI: {answer}
"""

# create a prompt example from above template
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

# now break our previous prompt into a prefix and suffix
# the prefix is our instructions
prefix = """The following are excerpts from conversations with an AI
assistant. The assistant is known for its humor and wit, providing
entertaining and amusing responses to users' questions. Here are some
examples:
"""
# and the suffix our user input and output indicator
suffix = """
User: {query}
AI: """

# now create the few-shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

In [9]:
print(few_shot_prompt_template.format(query="what is your favorite dish?"))

The following are excerpts from conversations with an AI
assistant. The assistant is known for its humor and wit, providing
entertaining and amusing responses to users' questions. Here are some
examples:



User: What's the weather like?
AI: It's raining cats and dogs, better bring an umbrella!



User: How old are you?
AI: Age is just a number, but I'm timeless.



User: what is your favorite dish?
AI: 


In [10]:
chain = LLMChain(prompt=few_shot_prompt_template, llm=llm_text_davinci, verbose=True)
chain.run("what is your favorite dish?")



> Entering new LLMChain chain...
Prompt after formatting:
The following are excerpts from conversations with an AI
assistant. The assistant is known for its humor and wit, providing
entertaining and amusing responses to users' questions. Here are some
examples:



User: What's the weather like?
AI: It's raining cats and dogs, better bring an umbrella!



User: How old are you?
AI: Age is just a number, but I'm timeless.



User: what is your favorite dish?
AI: 

> Finished chain.


' My favorite dish is anything that comes with a side of laughter!'

## Using Huggingface models

In [11]:
template = """Question: {question}

Answer: """
prompt = PromptTemplate(
    template=template,
    input_variables=['question']
)

# user question
question = "What is the capital city of France?"

In [12]:
# create prompt template > LLM chain
llm_chain = LLMChain(
    prompt=prompt,
    llm=hub_llm
)

# ask the user question about the capital of France
print(llm_chain.run(question))

paris


To ask multiple questions, we can either iterate through all questions one at a time or place all questions into a single prompt for more advanced LLMs. Let's start with the first approach:

In [13]:
qa = [
    {'question': "What is the capital city of France?"},
    {'question': "What is the largest mammal on Earth?"},
    {'question': "Which gas is most abundant in Earth's atmosphere?"},
    {'question': "What color is a ripe banana?"}
]
res = llm_chain.generate(qa)
for gen_list in res.generations:
    gen_text = gen_list[0].text
    print( gen_text )

paris
giraffe
nitrogen
yellow


### Text Translation

It is one of the great attributes of Large Language models that enables them to perform multiple tasks just by changing the prompt. We use the same llmCopy variable as defined before. However, pass a different prompt that asks for translating the query from a source_languageCopy to the target_languageCopy.

In [14]:
translation_template = "Translate the following text from {source_language} to {target_language}: {text}"
translation_prompt = PromptTemplate(input_variables=['source_language', 'target_language', 'text'], template=translation_template)

In [24]:
translation_chain = LLMChain(llm=llm_text_davinci, prompt=translation_prompt, verbose=True)

In [26]:
source_language = "English"
target_language = "Hindi"
text = "Hello"
translated_text = translation_chain.predict(source_language=source_language, target_language=target_language, text=text)
print (translated_text)



> Entering new LLMChain chain...
Prompt after formatting:
Translate the following text from English to Hindi: Hello

> Finished chain.


नमस्ते
